## Setting

In [1]:
import itertools
import json
import os
import pprint
import re
import sqlite3
import sys
from collections import Counter, defaultdict
from copy import deepcopy

import openpyxl
import pandas as pd

codefolder = (
    "C:/ProjectCollections/Programs/Australia_Cultural_Data_Engine/codes"
)

data_folder = "D:/Program_Data/Australia_Cultural_Data_Engine_Data/circusoz"

sys.path.append(codefolder)
from general import GeneralFunctions as gf
from general import JsonProcessing as jp
from general import MongoDBManipulation as mdb_manip

In [2]:
latest_working_wb = "Circus Oz_MasterList_working(as.of.14.Dec).xlsx"
latest_venue_wb = "Venue_consolidating doc(as.of.14.Dec).xlsx"
master_role_all = "23 Nov_Circus Oz_MasterList-RoleCategories_1978-2009_WorkingOut.xlsx"

color_mapping = {
    "theme_1": "cleaned",
    "theme_5": "AS",
    "theme_9": "FMP",
    "theme_4": "CO-vids",
}

In [3]:
circusoz_workbook_working = openpyxl.load_workbook(
    os.path.join(data_folder, latest_working_wb), data_only=True
)

In [4]:
circusoz_workbook_working.sheetnames

['EVENTS MASTER',
 'PERSON LIST MASTER ID LOOKUP',
 'EVENT ID LOOKUP',
 'Removed Events']

In [5]:
master_cols = defaultdict(list)
master_sheet_name = "EVENTS MASTER"
for c in circusoz_workbook_working[master_sheet_name][1]:
    if c.value != None:
        cell_color_dict = c.font.color.__dict__
        master_cols[
            f'{cell_color_dict.get("type")}_{cell_color_dict[cell_color_dict["type"]]}'
        ].append((c.col_idx, c.value))

for color_theme, cols in master_cols.items():
    print(color_theme, [col[1] for col in cols])

theme_6 ['(Missing.Cast)', '(Venue.Notes)', '(Estimated.Role.Note)', '(Record.Notes)']
theme_5 ['(AS list No.1)', '(AS.List.No.2)', 'Event.ID_(AS)', 'Event.ID.Alt_(AS)', 'Event.Name_(AS)', 'Alt.Event.Name_(AS)', 'Umbrella.Event_(AS)', 'Alt.Umbrella.Event_(AS)', 'First.Date_(AS)', 'Last.Date_(AS)', 'Venue.ID_(AS)', 'Venue.Name_(AS) ', 'Venue.Location_(AS)', 'Venue.Notes_(AS)', 'Further.Information_(AS)', 'Contributor.ID_(AS)', 'Gender_(AS)', 'Contributor.Function.1_(AS)', 'Contributor.Function.2_(AS)', 'Contributor.Function.3_(AS)']
theme_9 ['Season.Number_(FMP)', 'Category_(FMP)', 'Show.Name_(FMP)', 'Producer.Promoter_(FMP)', 'Date.from_(FMP)', 'date.to_(FMP)', 'Venue_(FMP)', 'City_(FMP)', 'Country_(FMP)', 'Venue.Type_(FMP)', 'Activity_(FMP)', 'Activity.Date_(FMP)', 'Activity.Note_(FMP)', 'Personnel.Number_(FMP)', 'Personnel.first.name_(FMP)', 'Personnel.Surname_(FMP)', 'Primary.Role_(FMP)', 'Secondary.role_(FMP)', 'Personnel.Note.1_(FMP)', 'Personnel.Note.2_(FMP)', 'Role.1.Binders.200

## 1. Venue

In [6]:
circusoz_workbook_venue = openpyxl.load_workbook(
    os.path.join(data_folder, latest_venue_wb), data_only=True
)
circusoz_workbook_venue.sheetnames

['Sheet1', 'Sheet2']

In [7]:
venue_cols = defaultdict(list)
venue_sheet_name = "Sheet1"
for c in circusoz_workbook_venue[venue_sheet_name][1]:
    if c.value != None:
        cell_color_dict = c.font.color.__dict__
        venue_cols[
            f'{cell_color_dict.get("type")}_{cell_color_dict[cell_color_dict["type"]]}'
        ].append(c.col_idx)

data = circusoz_workbook_venue[venue_sheet_name].values
venue_df = pd.DataFrame(data, columns=next(data)[0:],)

venue_cols.keys()

dict_keys(['theme_1', 'theme_5', 'theme_9', 'theme_4', 'theme_6'])

In [8]:
venue_collection = {}
table_cat = "venue"
for theme_name, data_source in color_mapping.items():
    if data_source == "cleaned":
        venue_table_name = table_cat
        venue_collection[table_cat] = (
            venue_df.iloc[:, map(lambda x: x - 1, venue_cols[theme_name])]
            .dropna(axis=1, how="all")
            .dropna(how="all")
        )
    else:
        venue_subset_cols = venue_df.columns[
            list(map(lambda x: x - 1, venue_cols[theme_name]))
        ].tolist()
        venue_table_name = f"{table_cat}_{data_source}"
        venue_collection[venue_table_name] = (
            venue_df[["EVENT.NUMBER", "VENUE.NAME"] + venue_subset_cols]
            .dropna(axis=1, how="all")
            .dropna(how="all", subset=venue_subset_cols,)
        )
    venue_collection[venue_table_name][
        ["EVENT.NUMBER", "VENUE.NAME"]
    ] = venue_collection[venue_table_name][["EVENT.NUMBER", "VENUE.NAME"]].fillna(
        "UNKNOWN"
    )

In [9]:
venue_collection["venue"]

,EVENT.NUMBER,VENUE.NAME,VENUE.ADDRESS,VENUE.TYPE
0,E0001,Pram Factory,"325 Drummond Street, Carlton, Victoria, 3053, ...",Other
1,E0002,Alexandra Gardens,"Melbourne, Victoria, 3000, Australia",Tent
2,E0003,Bonython Park,"Port Road, Adelaide, South Australia, 5000, Au...",Tent
3,E0004,UNKNOWN,"Melbourne, Victoria, 3000, Australia",Other
4,E0005,National Gallery of Victoria,"180 St Kilda Road, Melbourne, Victoria 3006, A...",Tent
...,...,...,...,...
475,E0471,UNKNOWN,None,None
476,E0472,UNKNOWN,None,None
477,E0473,UNKNOWN,None,None
478,E0474,UNKNOWN,None,None


## 2. Role Categories

In [10]:
circusoz_workbook_master_role = openpyxl.load_workbook(
    os.path.join(data_folder, master_role_all)
)

circusoz_workbook_master_role.sheetnames

['Events Master 1978 to 2009', 'Role Categories']

In [11]:
role_cat_cols = defaultdict(list)
role_cat_sheet_name = "Role Categories"
for c in circusoz_workbook_master_role[role_cat_sheet_name][1]:
    if c.value != None:
        cell_color_dict = c.font.color.__dict__
        role_cat_cols[
            f'{cell_color_dict.get("type")}_{cell_color_dict[cell_color_dict["type"]]}'
        ].append(c.col_idx)

data = circusoz_workbook_master_role[role_cat_sheet_name].values
role_cat_df = pd.DataFrame(data, columns=next(data)[0:],)

role_cat_cols.keys()

dict_keys(['theme_1', 'theme_9', 'theme_5', 'theme_4', 'rgb_FF7030A0'])

In [12]:
role_collection = {}
for color_cat in role_cat_cols.keys():
    if color_cat != "theme_1":
        role_cat_subset = (
            role_cat_df.iloc[
                :, min(role_cat_cols[color_cat]) - 1 : max(role_cat_cols[color_cat]) + 1
            ]
            .dropna(axis=1, how="all")
            .dropna(how="all")
        )
        role_subset_combined = pd.DataFrame()
        for idx in range(0, role_cat_subset.shape[1], 2):
            role_subset = role_cat_subset.iloc[:, idx : idx + 2].dropna(how="all")
            data_source = (
                color_mapping.get(color_cat)
                if color_mapping.get(color_cat)
                else "UNKNOWN"
            )
            role_subset["ori_attr_name"] = role_subset.columns[0]
            role_subset.columns = ["ori_attr_value", "ROLE.NUMBER", "ori_attr_name"]
            role_subset = role_subset[
                role_subset["ROLE.NUMBER"].apply(
                    lambda x: False if isinstance(x, str) and not x.isdigit() else True
                )
            ]
            role_subset["ROLE.NUMBER"] = role_subset["ROLE.NUMBER"].fillna(0).apply(int)
            role_subset_combined = pd.concat([role_subset_combined, role_subset])
        role_collection[f"role_{data_source}"] = role_subset_combined
    else:
        role_collection["role"] = (
            role_cat_df.iloc[
                :,
                min(role_cat_cols[color_cat]) - 1 : max(role_cat_cols[color_cat]) + 1,
            ]
            .dropna(axis=1, how="all")
            .dropna(how="all")
        )
        role_collection["role"] = pd.concat(
            [
                role_collection["role"],
                pd.DataFrame([[0, "Unknown"]], columns=["Number", "ROLE CATEGORIES"]),
            ],
            ignore_index=True,
        )
        role_collection["role"] = role_collection["role"][
            role_collection["role"]["Number"].apply(
                lambda x: True if isinstance(x, int) else False
            )
        ]
        role_collection["role"] = role_collection["role"].rename(
            {"Number": "ROLE.NUMBER"}, axis=1
        )
role_collection.keys()

dict_keys(['role', 'role_FMP', 'role_AS', 'role_CO-vids', 'role_UNKNOWN'])

In [13]:
role_collection["role"]

,ROLE.NUMBER,ROLE CATEGORIES,ROLE DEFINITIONS,Notes
0,1,Performer,All roles that refer to performing as part of ...,..
1,2,Musician,"All music-related performing roles, including ...",There is no consistence within or between data...
2,3,KeyCreative,Any role related to the creation of the shows ...,..
3,4,Crew,"Any production-related role, including all sta...",..
4,5,Administration,Any company or performance-related role note c...,..
5,6,Subsidiary,All company related roles that do not fall int...,..
6,7,Unknown,"All queries within the data ("","", ""?"" etc) or ...",None
7,8,Injured,All Injured/Workcover fields,None
13,0,Unknown,NaN,NaN


## 3. Event

In [14]:
"EVENT.NUMBER"
"PERSON.NUMBER"
master_cols_dict = {
    "EVENT": {
        "cleaned": [
            "EVENT.NUMBER",
            "TYPE",
            "TITLE",
            "UMBRELLA.EVENT",
            "UMBRELLA.EVENT.2",
            "DATE.FROM.(General)",
            "DATE.TO.(General)",
            "PERSON.NUMBER",
        ],
        "AS": [
            "EVENT.NUMBER",
            "Event.ID_(AS)",
            "Event.ID.Alt_(AS)",
            "Event.Name_(AS)",
            "Alt.Event.Name_(AS)",
            "Umbrella.Event_(AS)",
            "Alt.Umbrella.Event_(AS)",
            "First.Date_(AS)",
            "Last.Date_(AS)",
        ],
        "FMP": [
            "EVENT.NUMBER",
            "Season.Number_(FMP)",
            "Category_(FMP)",
            "Show.Name_(FMP)",
            "Producer.Promoter_(FMP)",
            "Date.from_(FMP)",
            "date.to_(FMP)",
            "Activity_(FMP)",
            "Activity.Date_(FMP)",
            "Activity.Note_(FMP)",
        ],
    },
    "PERSON": {
        "cleaned": [
            "PERSON.NUMBER",
            "FIRST.NAME",
            "SECOND.NAME",
            "COMBINED.NAME",
            "ALTERNATIVE.SPELLING (Discrepancy or inaccuracy)",
            # "Contributor.ID_(AS)",
            # "Personnel.Number_(FMP)",
        ],
        "AS": [
            "PERSON.NUMBER",
            "Contributor.ID_(AS)",
            "Contributor.Name(AS_&_Co-Clips)",
            "Gender_(AS)",
            "Contributor.Function.1_(AS)",
            "Contributor.Function.2_(AS)",
            "Contributor.Function.3_(AS)",
        ],
        "FMP": [
            "PERSON.NUMBER",
            "Personnel.Number_(FMP)",
            "Personnel.first.name_(FMP)",
            "Personnel.Surname_(FMP)",
            "Primary.Role_(FMP)",
            "Secondary.role_(FMP)",
            "Personnel.Note.1_(FMP)",
            "Personnel.Note.2_(FMP)",
            "Role.1.Binders.2003-04_(FMP)",
            "Role.2.Binders.2003-04_(FMP)",
            "Role.3.Binders.2003-04_(FMP)",
        ],
        "CO-clips": [
            "PERSON.NUMBER",
            "Video.Id.description_(CO-clips)",
            "Video.ID_(CO-vids)",
            "Source.3_(Co-clips)",
            "Contributor.Name(AS_&_Co-Clips)",
        ],
    },
}

In [15]:
data = circusoz_workbook_working[master_sheet_name].values
master_df = (
    pd.DataFrame(data, columns=next(data)[0:],).dropna(how="all")
    # .drop(["EVENT.NUMBER", "PERSON.NUMBER", "Contributor.ID", "Gender", "Contributor.ID_(AS)"], axis=1)
    .applymap(lambda x: None if x == "" or x == " " else x)
)

In [16]:
event_collection = {}
for data_source in master_cols_dict["EVENT"].keys():
    event_collection[f"event_{data_source}"] = (
        master_df[master_cols_dict["EVENT"][data_source]]
        .dropna(
            how="all",
            subset=set(master_cols_dict["EVENT"][data_source])
            - set(["EVENT.NUMBER", "PERSON.NUMBER"]),
        )
        .drop_duplicates()
    )

## 4. Person

In [17]:
person_collection = {}
for data_source in master_cols_dict["PERSON"].keys():
    person_collection[f"person_{data_source}"] = (
        master_df[master_cols_dict["PERSON"][data_source]]
        .dropna(
            how="all",
            subset=set(master_cols_dict["PERSON"][data_source])
            - set(["PERSON.NUMBER"]),
        )
        .dropna(
            how="all",
            subset=set(master_cols_dict["PERSON"][data_source])
            & set(["PERSON.NUMBER", "Contributor.ID_(AS)", "Personnel.Number_(FMP)"]),
        )
        .sort_values("PERSON.NUMBER")
        .drop_duplicates()
        .reset_index(drop=True)
    )
    if data_source == "CO-clips":
        person_collection[f"person_{data_source}"] = (
            person_collection[f"person_{data_source}"]
            .dropna(
                subset=[
                    "Video.Id.description_(CO-clips)",
                    "Video.ID_(CO-vids)",
                    "Source.3_(Co-clips)",
                ],
                how="all",
            )
            .sort_values("PERSON.NUMBER")
            .reset_index(drop=True)
        )
    elif data_source == "AS":
        person_collection[f"person_{data_source}"] = (
            person_collection[f"person_{data_source}"]
            .dropna(
                subset=[
                    "Contributor.ID_(AS)",
                    "Gender_(AS)",
                    "Contributor.Function.1_(AS)",
                    "Contributor.Function.2_(AS)",
                    "Contributor.Function.3_(AS)",
                ],
                how="all",
            )
            .sort_values("PERSON.NUMBER")
            .reset_index(drop=True)
        )

for data_source in master_cols_dict["PERSON"].keys():
    if data_source in ["AS", "FMP", "CO-clips"]:
        if data_source == "CO-clips":
            role_coll_name = "role_CO-vids"
        else:
            role_coll_name = f"role_{data_source}"
        person_role_link = pd.DataFrame()
        for attr_name in role_collection[role_coll_name]["ori_attr_name"].unique():
            person_role_link = pd.concat(
                [
                    person_role_link,
                    person_collection[f"person_{data_source}"][[attr_name]]
                    .dropna(subset=attr_name)
                    .reset_index()
                    .merge(
                        role_collection[role_coll_name][
                            role_collection[role_coll_name]["ori_attr_name"]
                            == attr_name
                        ],
                        left_on=attr_name,
                        right_on="ori_attr_value",
                        how="left",
                    )
                    .drop(["ori_attr_value", attr_name], axis=1),
                ]
            )
        person_collection[f"person_{data_source}"] = person_collection[
            f"person_{data_source}"
        ].merge(
            person_role_link.drop_duplicates().set_index("index"),
            left_index=True,
            right_index=True,
            how="left",
        )

## Import to DB

In [18]:
def table_import(import_df, created_table_name, primary_keys=[], foreign_keys=[]):

    """
    """

    sql_sent = []
    for col in import_df.columns:
        if col in primary_keys:
            sql_sent.append(f"`{col}` PRIMARY KEY")
        else:
            sql_sent.append(f"`{col}`")
    for local_key, ref_table, ref_key in foreign_keys:
        sql_sent.append(
            f"FOREIGN KEY(`{local_key}`) REFERENCES `{ref_table}`(`{ref_key}`)"
        )

    c.executescript(
        f"""
        DROP TABLE IF EXISTS `{created_table_name}`;
        CREATE TABLE {created_table_name} ({", ".join(sql_sent)});
    """
    )

    import_df.to_sql(name=created_table_name, con=conn, if_exists="append", index=False)

In [19]:
conn = sqlite3.connect(os.path.join(data_folder, "circuzOz_draft.db"))
c = conn.cursor()


#####
# Import venue relative tables
#####
table_import(
    import_df=venue_collection["venue"],
    created_table_name="VENUE",
    foreign_keys=[("EVENT.NUMBER", "EVENT", "EVENT.NUMBER")],
)

table_import(
    import_df=venue_collection["venue_FMP"],
    created_table_name="VENUE_FMP",
    primary_keys=[],
    foreign_keys=[("EVENT.NUMBER", "EVENT", "EVENT.NUMBER")],
)

table_import(
    import_df=venue_collection["venue_AS"],
    created_table_name="VENUE_AS",
    primary_keys=[],
    foreign_keys=[("EVENT.NUMBER", "EVENT", "EVENT.NUMBER")],
)

table_import(
    import_df=venue_collection["venue_CO-vids"],
    created_table_name="VENUE_CO",
    primary_keys=[],
    foreign_keys=[("EVENT.NUMBER", "EVENT", "EVENT.NUMBER")],
)
#####
# Import role relative tables
#####
table_import(
    import_df=role_collection["role"],
    created_table_name="ROLE",
    primary_keys=["ROLE.NUMBER"],
    foreign_keys=[],
)

table_import(
    import_df=role_collection["role_FMP"],
    created_table_name="ROLE_FMP",
    primary_keys=[],
    foreign_keys=[("ROLE.NUMBER", "ROLE", "ROLE.NUMBER")],
)

table_import(
    import_df=role_collection["role_AS"],
    created_table_name="ROLE_AS",
    primary_keys=[],
    foreign_keys=[("ROLE.NUMBER", "ROLE", "ROLE.NUMBER")],
)

table_import(
    import_df=role_collection["role_CO-vids"],
    created_table_name="ROLE_CO",
    primary_keys=[],
    foreign_keys=[("ROLE.NUMBER", "ROLE", "ROLE.NUMBER")],
)
#####
# Import person relative tables
#####
table_import(
    import_df=person_collection["person_cleaned"],
    created_table_name="PERSON",
    primary_keys=[],
    foreign_keys=[],
)

table_import(
    import_df=person_collection["person_AS"],
    created_table_name="PERSON_AS",
    primary_keys=[],
    foreign_keys=[
        ("PERSON.NUMBER", "PERSON", "PERSON.NUMBER"),
        ("ROLE.NUMBER", "ROLE", "ROLE.NUMBER"),
    ],
)
table_import(
    import_df=person_collection["person_FMP"],
    created_table_name="person_FMP",
    primary_keys=[],
    foreign_keys=[
        ("PERSON.NUMBER", "PERSON", "PERSON.NUMBER"),
        ("ROLE.NUMBER", "ROLE", "ROLE.NUMBER"),
    ],
)
table_import(
    import_df=person_collection["person_CO-clips"],
    created_table_name="PERSON_CO",
    primary_keys=[],
    foreign_keys=[
        ("PERSON.NUMBER", "PERSON", "PERSON.NUMBER"),
        ("ROLE.NUMBER", "ROLE", "ROLE.NUMBER"),
    ],
)
#####
# Import event relative tables
#####
table_import(
    import_df=event_collection["event_cleaned"],
    created_table_name="EVENT",
    primary_keys=[],
    foreign_keys=[("PERSON.NUMBER", "PERSON", "PERSON.NUMBER"),],
)

table_import(
    import_df=event_collection["event_AS"],
    created_table_name="EVENT_AS",
    foreign_keys=[("EVENT.NUMBER", "EVENT", "EVENT.NUMBER"),],
)
table_import(
    import_df=event_collection["event_FMP"],
    created_table_name="event_FMP",
    primary_keys=[],
    foreign_keys=[("EVENT.NUMBER", "EVENT", "EVENT.NUMBER"),],
)

In [20]:
#####
# Import views for loading
#####
with open('../../codes/circuzoz/circuzoz_CreateViews.sql', 'r') as f:
    sql_script = f.read()
    c.executescript(sql_script)

# Commit the changes
conn.commit()

# Close the connection
conn.close()